# AI実践演習(3)提出用
* 学籍番号：2364902
* 氏名：金 奎碩


---


## 演習(3)



### **取り組む問題の説明**
昔話題になったAlphaGoの深層強化学習（MCTS + Policy Network + Value Network + Self-Play）を利用して五目並べのAIを学習させてユーザと対戦を行う。
#### **概要・目的**
深層強化学習とモンテカルロ木探索（MCTS）を活用して、五目並べ（Gomoku）AI を構築する。このAIは、ニューラルネットワークを用いたポリシー・バリューネットワークを利用し、自己対局を通じて強化学習を行い、最適な手を探索できるようにする。そして、学習の量によって違うモデルで対戦を行うことでより強いAIを確認する。

##### 具体的な目的
1. ゲーム環境の構築：7×7の盤面で、5個の石が並んだら勝利となる五目並べ環境を構築。
2. ニューラルネットワークの設計：畳み込みニューラルネットワーク（CNN）を利用して、ポリシーと価値の予測を行うモデルを構築。
3. モンテカルロ木探索（MCTS）の実装：ニューラルネットワークの出力を基に、シミュレーションを行い、最適な手を探索
4. 自己対局によるデータ収集と学習：自己対局を通じてデータを収集し、ニューラルネットワークを学習。
5.  AIと対戦できるインターフェースの提供：学習したAIと対戦できる機能を実装。
#### **使用するデータセット**
自己対局によるデータ を生成し、それを学習する。外部のデータセットは使用せず、以下のようなデータを収集する予定。
* 各ゲームの 状態（盤面の配置）
* 各状態での モンテカルロ木探索（MCTS）による行動確率
* ゲームの結果（勝ち・負け・引き分け）

#### **学習するモデル**
* ニューラルネットワーク構造
畳み込みニューラルネットワーク（CNN）を利用したポリシー・バリューネットワーク を構築する。

  このネットワークは、以下の2つの役割を持っている。
  1. ポリシーネットワーク（Policy Network）：各状態における次の手の確率分布を予測。
  2. バリューネットワーク（Value Network）：その状態の勝率（評価値）を予測。
* モンテカルロ木探索（MCTS）
  * ノードの選択（Selection）：UCB1アルゴリズムを利用し、探索と活用のバランスを取る。
  * シミュレーション（Simulation）：ランダムプレイアウトの代わりに、ニューラルネットワークの出力を利用する。
  * バックプロパゲーション（Backpropagation）：各ノードのQ値と訪問回数を更新する。


#### **パラメータの決め方**
| **ハイパーパラメータ** | **値** | **説明** |
|-----------------|------|--------------|
| `BOARD_SIZE` | `7` | 盤面のサイズ（7×7） |
| `WIN_LENGTH` | `5` | 5個連続で並べたら勝利 |
| `c_puct` | `1.0` | MCTSの探索パラメータ |
| `n_simulations` | `1000` | 1手ごとのMCTSシミュレーション回数 |
| `learning_rate` | `0.001` | Adamオプティマイザの学習率 |
| `epochs` | `100` | 学習時のエポック数 |
| `batch_size` | `32` | ミニバッチサイズ |
| `num_self_play_games` | `20` | 自己対局の回数 |

#### **参考サイトのURL（あれば）**
1. https://discovery.ucl.ac.uk/id/eprint/10045895/1/agz_unformatted_nature.pdf
2. https://qiita.com/toyohisa/items/e9f218909214c3a98ce2?utm_source=chatgpt.com
3. https://qiita.com/pocokhc/items/392a7f89c79f5e1e6bca?utm_source=chatgpt.com
4. https://www.brainpad.co.jp/doors/contents/01_tech_2018-04-05-163000/?utm_source=chatgpt.com

### 1. ゲーム環境とヘルパー関数

In [9]:
BOARD_SIZE = 7
WIN_LENGTH = 5

def check_win(board, player):
    # 指定されたプレイヤーの石が横、縦、または斜めに WIN_LENGTH 以上連続しているか確認
    for i in range(BOARD_SIZE):
        for j in range(BOARD_SIZE):
            if board[i, j] == player:
                # 横のチェック
                if j <= BOARD_SIZE - WIN_LENGTH and all(board[i, j + k] == player for k in range(WIN_LENGTH)):
                    return True
                # 縦のチェック
                if i <= BOARD_SIZE - WIN_LENGTH and all(board[i + k, j] == player for k in range(WIN_LENGTH)):
                    return True
                # 斜め（右下）
                if i <= BOARD_SIZE - WIN_LENGTH and j <= BOARD_SIZE - WIN_LENGTH and all(board[i + k, j + k] == player for k in range(WIN_LENGTH)):
                    return True
                # 斜め（左下）
                if i >= WIN_LENGTH - 1 and j <= BOARD_SIZE - WIN_LENGTH and all(board[i - k, j + k] == player for k in range(WIN_LENGTH)):
                    return True
    return False

def get_valid_moves(state):
    board, _ = state
    valid = []
    for i in range(BOARD_SIZE):
        for j in range(BOARD_SIZE):
            if board[i, j] == 0:
                valid.append(i * BOARD_SIZE + j)
    return valid

def get_next_state(state, action):
    board, player = state
    new_board = board.copy()
    row = action // BOARD_SIZE
    col = action % BOARD_SIZE
    new_board[row, col] = player
    return (new_board, -player)

def serialize_state(state):
    board, player = state
    # 現在のプレイヤー情報を1バイトに保存
    return board.tobytes() + (b'\x00' if player == 1 else b'\x01')

def get_game_result(state):
    board, player = state
    # 最後に置いた石が勝利条件を満たすか確認
    if check_win(board, player):  # <- 元の -player を player に変更
        return 1  # 勝利

    if len(get_valid_moves(state)) == 0:
        return 0  # 引き分け

    return None  # ゲーム進行中


def state_to_tensor(state):
    board, player = state
    tensor = np.zeros((2, BOARD_SIZE, BOARD_SIZE), dtype=np.float32)
    tensor[0] = (board == player).astype(np.float32)
    tensor[1] = (board == -player).astype(np.float32)
    return torch.tensor(tensor)

def print_board(board):
    for i in range(BOARD_SIZE):
        line = ""
        for j in range(BOARD_SIZE):
            if board[i, j] == 1:
                line += "X "
            elif board[i, j] == -1:
                line += "O "
            else:
                line += ". "
        print(line)
    print()  # 空行を追加


### 2. ニューラルネットワークモデル（Policy & Value）

In [10]:
import torch.nn as nn

class GomokuNet(nn.Module):
    def __init__(self):
        super(GomokuNet, self).__init__()
        self.conv1 = nn.Conv2d(2, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * BOARD_SIZE * BOARD_SIZE, 128)
        self.fc_policy = nn.Linear(128, BOARD_SIZE * BOARD_SIZE)
        self.fc_value = nn.Linear(128, 1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = x.view(-1, 64 * BOARD_SIZE * BOARD_SIZE)
        x = F.relu(self.fc1(x))
        policy = self.fc_policy(x)
        value = torch.tanh(self.fc_value(x))
        return policy, value

### 3. MCTSの実装

In [11]:
class MCTS:
    def __init__(self, net, c_puct=1.0, n_simulations=1000):
        self.net = net
        self.c_puct = c_puct
        self.n_simulations = n_simulations
        self.Qsa = {}  # (state, action)ごとの Q値
        self.Nsa = {}  # (state, action)ごとの訪問回数
        self.Ns = {}   # stateごとの訪問回数
        self.Ps = {}   # stateごとの事前確率（ニューラルネットワークの予測）
        self.Es = {}   # stateの終了状態と結果
        self.Vs = {}   # stateで可能な行動のリスト

    def get_action_probs(self, state, temp=1):
        for _ in range(self.n_simulations):
            self.search(state)
        s = serialize_state(state)
        counts = [self.Nsa.get((s, a), 0) for a in range(BOARD_SIZE * BOARD_SIZE)]
        if temp == 0:
            bestA = np.argmax(counts)
            probs = [0] * (BOARD_SIZE * BOARD_SIZE)
            probs[bestA] = 1
            return probs
        counts = [x ** (1. / temp) for x in counts]
        total = float(sum(counts))
        probs = [x / total for x in counts]
        return probs

    def search(self, state):
        s = serialize_state(state)
        result = get_game_result(state)
        if result is not None:
            # 終端状態: 現在のプレイヤーの視点で結果を返す
            return -result

        if s not in self.Ps:
            # リーフノードに到達: ニューラルネットワークで評価およびポリシー予測
            valid_moves = get_valid_moves(state)
            board_tensor = state_to_tensor(state).unsqueeze(0)  # バッチ次元を追加
            self.net.eval()
            with torch.no_grad():
                logits, value = self.net(board_tensor)
            probs = F.softmax(logits, dim=1).cpu().numpy().flatten().tolist()
            # 無効な手の確率を0に設定
            for a in range(BOARD_SIZE * BOARD_SIZE):
                if a not in valid_moves:
                    probs[a] = 0
            sum_probs = sum(probs)
            if sum_probs > 0:
                probs = [p / sum_probs for p in probs]
            else:
                # すべての確率が0の場合、有効な手に対して均等分布を適用
                probs = [1 / len(valid_moves) if a in valid_moves else 0 for a in range(BOARD_SIZE * BOARD_SIZE)]
            self.Ps[s] = probs
            self.Vs[s] = valid_moves
            self.Ns[s] = 0
            return -value.item()

        # 選択: UCBを使用して最適な行動を選択
        best_ucb = -float('inf')
        best_action = -1
        for a in self.Vs[s]:
            if (s, a) in self.Qsa:
                ucb = self.Qsa[(s, a)] + self.c_puct * self.Ps[s][a] * np.sqrt(self.Ns[s]) / (1 + self.Nsa[(s, a)])
            else:
                ucb = self.c_puct * self.Ps[s][a] * np.sqrt(self.Ns[s] + 1e-8)
            if ucb > best_ucb:
                best_ucb = ucb
                best_action = a
        a = best_action
        next_state = get_next_state(state, a)
        v = self.search(next_state)
        # バックアップ: Q, Nの更新
        if (s, a) in self.Qsa:
            self.Qsa[(s, a)] = (self.Nsa[(s, a)] * self.Qsa[(s, a)] + v) / (self.Nsa[(s, a)] + 1)
            self.Nsa[(s, a)] += 1
        else:
            self.Qsa[(s, a)] = v
            self.Nsa[(s, a)] = 1
        self.Ns[s] += 1
        return -v


### 4. 自己対局とネットワーク学習

In [12]:
def self_play_game(net, n_simulations=1000):
    mcts = MCTS(net, n_simulations=n_simulations)
    train_examples = []
    state = (np.zeros((BOARD_SIZE, BOARD_SIZE), dtype=np.int8), 1)
    while True:
        # 序盤は探索の温度を高く設定 (temp=1)、終盤は決定論的に (temp=0)
        temp = 1 if len(train_examples) < 10 else 0
        action_probs = mcts.get_action_probs(state, temp=temp)
        train_examples.append((state, action_probs, None))
        # 行動選択: 確率に基づいてサンプリング
        action = np.random.choice(range(BOARD_SIZE * BOARD_SIZE), p=action_probs)
        state = get_next_state(state, action)
        result = get_game_result(state)
        if result is not None:
            # ゲーム終了: 各手に結果を付与 (プレイヤー交代に伴う符号の変更)
            return [(s, pi, result * ((-1) ** i)) for i, (s, pi, _) in enumerate(train_examples)]

def train_network(net, examples, epochs=10, batch_size=64, lr=0.001):
    optimizer = optim.Adam(net.parameters(), lr=lr)
    net.train()
    for epoch in range(epochs):
        random.shuffle(examples)
        for i in range(0, len(examples), batch_size):
            batch = examples[i:i + batch_size]
            states, target_pis, target_vs = zip(*batch)
            state_tensors = torch.stack([state_to_tensor(s) for s in states])
            target_pis = torch.tensor(target_pis, dtype=torch.float32)
            target_vs = torch.tensor(target_vs, dtype=torch.float32).view(-1, 1)
            optimizer.zero_grad()
            pred_logits, pred_vs = net(state_tensors)
            loss_v = F.mse_loss(pred_vs, target_vs)
            loss_pi = -torch.mean(torch.sum(target_pis * F.log_softmax(pred_logits, dim=1), dim=1))
            loss = loss_v + loss_pi
            loss.backward()
            optimizer.step()
        print(f"Epoch {epoch + 1} completed.")

### 5. モデルの保存/読み込みとユーザー対戦

In [13]:
def play_against_ai(net, n_simulations=1000):
    state = (np.zeros((BOARD_SIZE, BOARD_SIZE), dtype=np.int8), 1)
    while True:
        board, player = state
        print_board(board)
        if player == 1:
            # ユーザーのターン
            move = input("行と列をスペースで区切って入力してください (例: 2 3): ")
            try:
                row, col = map(int, move.split())
            except:
                print("無効な入力です。もう一度入力してください。")
                continue
            action = row * BOARD_SIZE + col
            if board[row, col] != 0:
                print("その位置はすでに使用されています。他の場所を選択してください。")
                continue
        else:
            # AIのターン: MCTSを使用して決定
            mcts = MCTS(net, n_simulations=n_simulations)
            action_probs = mcts.get_action_probs(state, temp=0)
            action = np.argmax(action_probs)
            ai_row, ai_col = action // BOARD_SIZE, action % BOARD_SIZE
            print(f"AIが {ai_row}, {ai_col} の位置に置きました。")
        state = get_next_state(state, action)
        result = get_game_result(state)
        if result is not None:
            print_board(state[0])
            if result == 0:
                print("引き分けです。")
            else:
                # 最後に石を置いたプレイヤーが勝利した状態なので、
                # 現在のターン (player) が 1 の場合、相手が勝利したことになる
                if player == 1:
                    print("敗北しました。")
                else:
                    print("勝利しました。")
            break

### 6. 全体の実行

In [14]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import random

if __name__ == "__main__":
    net = GomokuNet()

    # 自己対局による学習データの収集 (例: 50ゲーム)
    examples = []
    num_self_play_games = 50
    print("自己対局を開始します...")
    for i in range(num_self_play_games):
        game_examples = self_play_game(net, n_simulations=1000)
        examples.extend(game_examples)
        print(f"ゲーム {i+1} 完了。")

    # 収集したデータを用いてネットワークを学習
    print("ネットワークの学習を開始します...")
    train_network(net, examples, epochs=100, batch_size=32, lr=0.001)

    # 学習済みモデルを保存
    torch.save(net.state_dict(), "omok_model1.pth")
    print("モデルが保存されました: omok_model1.pth")

    # 保存したモデルの読み込み (必要に応じて)
    # net.load_state_dict(torch.load("omok_model.pth"))

    # ユーザーとAIの対戦
    print("ユーザーとAIの対戦を開始します。")
    play_against_ai(net, n_simulations=1000)


自己対局を開始します...
ゲーム 1 完了。
ゲーム 2 完了。
ゲーム 3 完了。
ゲーム 4 完了。
ゲーム 5 完了。
ゲーム 6 完了。
ゲーム 7 完了。
ゲーム 8 完了。
ゲーム 9 完了。
ゲーム 10 完了。
ゲーム 11 完了。
ゲーム 12 完了。
ゲーム 13 完了。
ゲーム 14 完了。
ゲーム 15 完了。
ゲーム 16 完了。
ゲーム 17 完了。
ゲーム 18 完了。
ゲーム 19 完了。
ゲーム 20 完了。
ゲーム 21 完了。
ゲーム 22 完了。
ゲーム 23 完了。
ゲーム 24 完了。
ゲーム 25 完了。
ゲーム 26 完了。
ゲーム 27 完了。
ゲーム 28 完了。
ゲーム 29 完了。
ゲーム 30 完了。
ゲーム 31 完了。
ゲーム 32 完了。
ゲーム 33 完了。
ゲーム 34 完了。
ゲーム 35 完了。
ゲーム 36 完了。
ゲーム 37 完了。
ゲーム 38 完了。
ゲーム 39 完了。
ゲーム 40 完了。
ゲーム 41 完了。
ゲーム 42 完了。
ゲーム 43 完了。
ゲーム 44 完了。
ゲーム 45 完了。
ゲーム 46 完了。
ゲーム 47 完了。
ゲーム 48 完了。
ゲーム 49 完了。
ゲーム 50 完了。
ネットワークの学習を開始します...
Epoch 1 completed.
Epoch 2 completed.
Epoch 3 completed.
Epoch 4 completed.
Epoch 5 completed.
Epoch 6 completed.
Epoch 7 completed.
Epoch 8 completed.
Epoch 9 completed.
Epoch 10 completed.
Epoch 11 completed.
Epoch 12 completed.
Epoch 13 completed.
Epoch 14 completed.
Epoch 15 completed.
Epoch 16 completed.
Epoch 17 completed.
Epoch 18 completed.
Epoch 19 completed.
Epoch 20 completed.
Epoch 21 completed.
Epoch 22 comple

### AIと対戦(num_self_play_games = 50, n_simulations = 500）

In [15]:
    # 保存したモデルの読み込み (必要に応じて)
    net.load_state_dict(torch.load("omok_model.pth"))

    # ユーザーとAIの対戦
    print("ユーザーとAIの対戦を開始します。")
    play_against_ai(net, n_simulations=1000)

<ipython-input-15-c35433139b6b>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load("omok_model.pth"))


ユーザーとAIの対戦を開始します。
. . . . . . . 
. . . . . . . 
. . . . . . . 
. . . . . . . 
. . . . . . . 
. . . . . . . 
. . . . . . . 

行と列をスペースで区切って入力してください (例: 2 3): 0 1
. X . . . . . 
. . . . . . . 
. . . . . . . 
. . . . . . . 
. . . . . . . 
. . . . . . . 
. . . . . . . 

AIが 4, 5 の位置に置きました。
. X . . . . . 
. . . . . . . 
. . . . . . . 
. . . . . . . 
. . . . . O . 
. . . . . . . 
. . . . . . . 

行と列をスペースで区切って入力してください (例: 2 3): 0 2
. X X . . . . 
. . . . . . . 
. . . . . . . 
. . . . . . . 
. . . . . O . 
. . . . . . . 
. . . . . . . 

AIが 5, 0 の位置に置きました。
. X X . . . . 
. . . . . . . 
. . . . . . . 
. . . . . . . 
. . . . . O . 
O . . . . . . 
. . . . . . . 

行と列をスペースで区切って入力してください (例: 2 3): 0 3
. X X X . . . 
. . . . . . . 
. . . . . . . 
. . . . . . . 
. . . . . O . 
O . . . . . . 
. . . . . . . 

AIが 2, 3 の位置に置きました。
. X X X . . . 
. . . . . . . 
. . . O . . . 
. . . . . . . 
. . . . . O . 
O . . . . . . 
. . . . . . . 

行と列をスペースで区切って入力してください (例: 2 3): 0 4
. X X X X . . 
. . . . . . . 
. . . 

### AIと対戦(num_self_play_games = 50, n_simulations = 1000）

In [16]:
    # 保存したモデルの読み込み (必要に応じて)
    net.load_state_dict(torch.load("omok_model1.pth"))

    # ユーザーとAIの対戦
    print("ユーザーとAIの対戦を開始します。")
    play_against_ai(net, n_simulations=1000)

<ipython-input-16-9f4ad595b7ff>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load("omok_model1.pth"))


ユーザーとAIの対戦を開始します。
. . . . . . . 
. . . . . . . 
. . . . . . . 
. . . . . . . 
. . . . . . . 
. . . . . . . 
. . . . . . . 

行と列をスペースで区切って入力してください (例: 2 3): 3 3
. . . . . . . 
. . . . . . . 
. . . . . . . 
. . . X . . . 
. . . . . . . 
. . . . . . . 
. . . . . . . 

AIが 3, 1 の位置に置きました。
. . . . . . . 
. . . . . . . 
. . . . . . . 
. O . X . . . 
. . . . . . . 
. . . . . . . 
. . . . . . . 

行と列をスペースで区切って入力してください (例: 2 3): 3 2
. . . . . . . 
. . . . . . . 
. . . . . . . 
. O X X . . . 
. . . . . . . 
. . . . . . . 
. . . . . . . 

AIが 4, 4 の位置に置きました。
. . . . . . . 
. . . . . . . 
. . . . . . . 
. O X X . . . 
. . . . O . . 
. . . . . . . 
. . . . . . . 

行と列をスペースで区切って入力してください (例: 2 3): 2 3
. . . . . . . 
. . . . . . . 
. . . X . . . 
. O X X . . . 
. . . . O . . 
. . . . . . . 
. . . . . . . 

AIが 5, 1 の位置に置きました。
. . . . . . . 
. . . . . . . 
. . . X . . . 
. O X X . . . 
. . . . O . . 
. O . . . . . 
. . . . . . . 

行と列をスペースで区切って入力してください (例: 2 3): 3 4
. . . . . . . 
. . . . . . . 
. . . 

### 考察
実際に対戦を行ってみたが思ったより正確に作ることはできなかった。

その理由としてはnum_self_play_games = 50, n_simulations = 1000の場合学習させるのに1時間がかかってる。しかし、五目並べの場合場合の数がたくさん存在するので多くの場合を学習させるためにはたくさんの学習が必要である。

それに加えて、AlphaGoZeroの場合はスーパーコンピューターを利用して学習させるのでスーパーコンピューターを利用することはできないので一番五目並べが生じない位置に置くだけのAIになったと思う。





---


## 参考文献
* DeepMind、「Mastering the game of Go without human knowledge」、UCL Discovery、2017年、https://discovery.ucl.ac.uk/id/eprint/10045895/1/agz_unformatted_nature.pdf、 （参照：2025-02-10）

* Toyohisa、「モンテカルロ木探索による五目並べAI #Python」、Qiita、更新日不明、https://qiita.com/toyohisa/items/e9f218909214c3a98ce2?utm_source=chatgpt.com、 （参照：2025-02-10）

* Pocokhc、「【強化学習】モンテカルロ木探索を解説・実装」、Qiita、更新日不明、https://qiita.com/pocokhc/items/392a7f89c79f5e1e6bca?utm_source=chatgpt.com、 （参照：2025-02-10）

* BrainPad、「AlphaGoZeroでも重要な技術要素！ モンテカルロ木探索の入門」、BrainPad、2018年4月5日、https://www.brainpad.co.jp/doors/contents/01_tech_2018-04-05-163000/?utm_source=chatgpt.com、 （参照：2025-02-10）




---


## 講義の感想

### 良かった点、興味を持った内容など
ニュースやネットでしか見てないAIを実際に実装することでAIに関する理解が深まった。

そして、一番興味深く思った学習方法は強化学習である。

AIが人間のように試行錯誤を繰り返しながら学習することと実際に目に見える所の学習ができる点である。

### 難しかった点、講義の運営方法に対する要望など
情報工学は1年生からプログラミング演習でC言語と今回はJAVAのプログラミングの演習を行ったがPythonはしっかり学んでないため予め知識がない人はコードの理解が難しいと思う。

現在はパラメータの理解と学習方法を理解することはできるが、細かいところにわからないことが多い。